In [ ]:
import math
from numpy import*
def getOXmatrix(i): # создает марицу поворота относительно OX
    m = numpy.zeros((3,3))
    m[0][0] = 1
    m[1][1] = m[2][2] = math.cos(i)
    m[1][2] = (-1)*math.sin(i)
    m[2][1] = math.sin(i)
    return m
def createVec(x,y,z):
    v = zeros((3,1))
    v[0][0] = x
    v[1][0] = y
    v[2][0] = z
    return v

# body
# ввод данных
print ("Введите 6 элементов Кеплера:")
while True:
    print ("1) Большая полуось (км): ", end = "")
    a = float(input())
    if a>0:
        break
while True:
    print ("2) Эксцентриситет: ", end = "")
    e = float(input())
    if e>=0 and e<1:
        break
print ("3) Наклон (в градусах): ", end = "")
i = (float(input())%360)/180*math.pi
print ("4) Долгота восходящего угла (в градусах): ", end = "")
d = (float(input())%360)/180*math.pi
print ("5) Аргумент перицентра (в градусах): ", end = "")
w = (float(input())%360)/180*math.pi
print ("6) Средняя аномалия (в градусах): ", end = "")
M = (float(input())%360)/180*math.pi

# вычисление позиции в двумерном пространстве
E = M # пока пусть так
f = float(a*e) # расстояние от центра эллипса до точки фокуса
b = math.sqrt(a*a - f*f) # величина малой полуоси
x = (a * math.cos(E))
y = math.sqrt((b*b)-((x*x*b*b)/(a*a)))
x = x - f
z = 0.0
vec = createVec(x,y,z)
# вычисление координат с учетом наклона (i)
chMat = getOXmatrix(i)
total = chMat.dot(vec)
print (total)
#print("Тело расположено в точке S - ",x,y)
